In [8]:
import pandas as pd

---

#### Inflation


Inflation tables updated monthly at: https://bbs.gov.bd/site/page/29b379ff-7bac-41d9-b321-e41929bab4a1/-

- Get most recent year month from `Consumer Price Index (CPI) and Inflation Rate (IR)` section:
- data table in pdf format, so use `tabula` to extract data

In [1]:
import tabula

In [3]:
dataset = 'https://bbs.portal.gov.bd/sites/default/files/files/bbs.portal.gov.bd/page/9ead9eb1_91ac_4998_a1a3_a5caf4ddc4c6/2023-09-25-08-25-0a59c12c663e2cc22c9be416bcb1fa82.pdf'

In [6]:
# Read the PDF and extract tables into a list of DataFrames
table = tabula.read_pdf(dataset, pages=1, multiple_tables=False)

The output file is empty.


In [7]:
table

[]

In [4]:


# Assuming the table you want is the first table in the PDF, you can access it like this
# Replace 0 with the appropriate index if the table is at a different position
df = tables[0]

# Display the DataFrame
print(df)

IndexError: list index out of range

---

### World Bank

**Country Page:** https://data.worldbank.org/country/bangladesh

Data Catalog: https://api.worldbank.org/v2/en/country/BGD?downloadformat=csv

In [14]:
import requests
import zipfile
import io
import pandas as pd

# URL of the zip file
url = 'https://api.worldbank.org/v2/en/country/BGD?downloadformat=csv'

# Send an HTTP request and get the zip file content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Extract the CSV file starting with 'API' from the zip file
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        csv_filename = None
        for file_name in zip_ref.namelist():
            if file_name.startswith('API') and file_name.endswith('.csv'):
                csv_filename = file_name
                break
        
        if csv_filename:
            # Extract the CSV file
            zip_ref.extract(csv_filename)
            csv_file = csv_filename
        else:
            print('No CSV file starting with "API" found in the zip file.')
            exit(1)
else:
    print('Failed to download the zip file.')
    exit(1)

ParserError: Error tokenizing data. C error: Expected 3 fields in line 5, saw 68


In [28]:
# Read the CSV file into a pandas DataFrame, ignore first two columns
raw_data = pd.read_csv(csv_file, header=2, index_col=False)
raw_data = raw_data.drop(raw_data.columns[[0, 1]], axis=1)  # drop country name & code
raw_data = raw_data.dropna(axis=1, how='all') # drop any columns that only contain missing values
raw_data.head(5)

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"Internally displaced persons, total displaced ...",VC.IDP.TOCV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,426000.000000,426000.000000,426000.000000,426000.000000,432000.000000,426000.000000,427000.000000,427000.000000,427000.000000,427000.0
1,Voice and Accountability: Standard Error,VA.STD.ERR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.110641,0.127796,0.129927,0.125566,0.126059,0.128895,0.123450,0.127924,0.130795,NaN
2,Voice and Accountability: Number of Sources,VA.NO.SRC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.000000,12.000000,12.000000,12.000000,12.000000,12.000000,11.000000,10.000000,10.000000,NaN
3,High-technology exports (% of manufactured exp...,TX.VAL.TECH.MF.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.299089,NaN,0.307867,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Export value index (2015 = 100),TX.VAL.MRCH.XD.WD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.915593,93.904913,100.000000,107.769398,110.723624,121.227906,121.491474,103.788540,136.581796,NaN


---

#### Extract Series

Create a function to help search dataframe

- create function that takes a loose data series we want to extract in the form of a list of strings, and searches `Indicator Name` column, and returns the rows of any relevant series we might want to extract

In [ ]:
def filter_series(dataframe, keywords):
    """
    Filter a DataFrame based on a list of keywords present in the 'Indicator Name' column.

    Parameters:
    - dataframe (pd.DataFrame): The DataFrame to be filtered.
    - keywords (list): A list of keywords to search for in a case-insensitive manner.

    Returns:
    - pd.DataFrame: A filtered DataFrame containing rows where the 'Indicator Name' column
      contains all the specified keywords.

    Example:
    >>> keywords = ['population', 'total']
    >>> filtered_data = filter_series(raw_data, keywords)
    >>> print(filtered_data)
       Indicator Name Indicator Code      1960      1961      1962      1963      1964
    0  Population, total   SP.POP.TOTL  50396429  51882769  53461661  55094115  56774465
    """
    # Convert keywords to lowercase for case-insensitive search
    keywords_lower = [keyword.lower() for keyword in keywords]
    
    # Initialize a mask with True for all rows
    mask = pd.Series(True, index=dataframe.index)
    
    # Apply the mask for each keyword
    for keyword in keywords_lower:
        mask &= dataframe['Indicator Name'].str.lower().str.contains(keyword)
    
    # Filter the DataFrame using the combined mask
    filtered_data = dataframe[mask]
    
    return filtered_data

##### Population

In [65]:
filter_series(raw_data, ['Population', 'total']).head()


,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
26,Urban population (% of total population),SP.URB.TOTL.IN.ZS,5.135000e+00,5.278000e+00,5.498000e+00,5.727000e+00,5.964000e+00,6.211000e+00,6.467000e+00,6.733000e+00,...,3.276200e+01,3.353500e+01,3.430800e+01,3.508300e+01,3.585800e+01,3.663200e+01,3.740500e+01,3.817700e+01,3.894600e+01,3.971100e+01
29,"Population, male (% of total population)",SP.POP.TOTL.MA.ZS,5.190252e+01,5.187827e+01,5.185654e+01,5.184286e+01,5.183347e+01,5.182633e+01,5.182088e+01,5.181263e+01,...,4.985193e+01,4.980310e+01,4.975974e+01,4.972908e+01,4.970669e+01,4.967779e+01,4.964116e+01,4.961369e+01,4.959846e+01,4.957124e+01
30,"Population, total",SP.POP.TOTL,5.039643e+07,5.188277e+07,5.346166e+07,5.509412e+07,5.677446e+07,5.850016e+07,6.026526e+07,6.210449e+07,...,1.540301e+08,1.559613e+08,1.578300e+08,1.597846e+08,1.617940e+08,1.636840e+08,1.655162e+08,1.674210e+08,1.693563e+08,1.711864e+08
40,Population ages 15-64 (% of total population),SP.POP.1564.TO.ZS,5.472769e+01,5.451445e+01,5.425726e+01,5.393275e+01,5.355325e+01,5.322510e+01,5.295755e+01,5.270504e+01,...,6.380357e+01,6.440960e+01,6.499142e+01,6.555376e+01,6.609745e+01,6.658262e+01,6.699931e+01,6.738486e+01,6.771990e+01,6.798642e+01
43,Population ages 0-14 (% of total population),SP.POP.0014.TO.ZS,4.226348e+01,4.248862e+01,4.274926e+01,4.306812e+01,4.343385e+01,4.374363e+01,4.398707e+01,4.421036e+01,...,3.153206e+01,3.086886e+01,3.020124e+01,2.952014e+01,2.883163e+01,2.817087e+01,2.755682e+01,2.698218e+01,2.645298e+01,2.596916e+01


In [69]:
# We know 'Population, total' is in row index 30 of master dataframe, so extract this into its own df
df_wide = raw_data.iloc[[30]].reset_index(drop=True)
df_wide

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"Population, total",SP.POP.TOTL,50396429.0,51882769.0,53461661.0,55094115.0,56774465.0,58500159.0,60265259.0,62104488.0,...,154030139.0,155961299.0,157830000.0,159784568.0,161793964.0,163683958.0,165516222.0,167420951.0,169356251.0,171186372.0


In [78]:
# Convert the DataFrame from wide to long format
df_long = pd.melt(df_wide, id_vars=['Indicator Name', 'Indicator Code'], var_name='Year', value_name='Value')

# create new df with only `Year` and `Value` columns
df = df_long[['Year', 'Value']]

# rename cols
df.columns = ['date', 'value']

# convert `date` column to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y')

# convert `value` col to int
df['value'] = df['value'].astype(int)

/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_53519/3717464359.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format='%Y')
/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_53519/3717464359.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['value'] = df['value'].astype(int)


In [84]:
# export to csv and json
df.to_csv('data/popu.csv', index=False)
df.to_json('data/popu.json', orient='records')

---

#### Unemployment

In [89]:
filter_series(raw_data, ['Unemployment', 'total'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
63,Adequacy of unemployment benefits and ALMP (% ...,per_lm_alllm.adq_pop_tot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.881,10.268,10.666,11.051,12.219,12.293,12.366,14.383,13.772,12.928
804,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.430,NaN,NaN,4.350,4.370,NaN,NaN,NaN,NaN,NaN
806,Unemployment with intermediate education (% of...,SL.UEM.INTM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.980,NaN,NaN,9.710,8.480,NaN,NaN,NaN,NaN,NaN
861,Benefit incidence of unemployment benefits and...,per_lm_alllm.ben_q1_tot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1125,Unemployment with advanced education (% of tot...,SL.UEM.ADVN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.000,NaN,NaN,9.030,10.740,NaN,NaN,NaN,NaN,NaN
1126,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.NE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.880,NaN,NaN,11.370,12.760,NaN,NaN,NaN,NaN,NaN
1223,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.430,4.393,4.371,4.350,4.370,4.380,4.382,5.209,5.097,4.699
1226,Unemployment with basic education (% of total ...,SL.UEM.BASC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.820,NaN,NaN,3.230,3.450,NaN,NaN,NaN,NaN,NaN


In [146]:
def extract_series(row_index):
    '''
    pass in index of row to extract from `raw_data` and return a df in long format
    '''
    # extract row index 1223
    df_wide = raw_data.iloc[[row_index]].reset_index(drop=True)

    print(df_wide.iloc[0, 0], '\n') # print the first value of the first column
    print(df_wide)

    # drop NaN columns
    df_wide = df_wide.dropna(axis=1, how='all')

    # Convert the DataFrame from wide to long format
    df_long = pd.melt(df_wide, id_vars=['Indicator Name', 'Indicator Code'], var_name='date', value_name='value')

    # create new df with only `date` and `value` columns
    df = df_long[['date', 'value']]

    # convert `date` column to datetime (using .loc to avoid copy warning)
    df.loc[:, 'date'] = pd.to_datetime(df['date'], format='%Y').dt.strftime("%Y-%m-%d")

    print(df.info())

    return df

In [147]:
df = extract_series(1223)

Unemployment, total (% of total labor force) (modeled ILO estimate) 

                                      Indicator Name  Indicator Code  1960  \
0  Unemployment, total (% of total labor force) (...  SL.UEM.TOTL.ZS   NaN   

   1961  1962  1963  1964  1965  1966  1967  ...  2013   2014   2015  2016  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  4.43  4.393  4.371  4.35   

   2017  2018   2019   2020   2021   2022  
0  4.37  4.38  4.382  5.209  5.097  4.699  

[1 rows x 65 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    32 non-null     object 
 1   value   32 non-null     float64
dtypes: float64(1), object(1)
memory usage: 640.0+ bytes
None


In [148]:
# export to csv and json
df.to_csv('data/unem.csv', index=False)
df.to_json('data/unem.json', orient='records')

#### Inactivity

#### Employment Rate

In [177]:
filter_series(raw_data, ['employment', 'population ratio', '15+', 'total'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
684,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.175,54.006,53.829,53.650,55.781,55.911,56.019,55.007,55.277,55.993
1146,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.NE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.650,NaN,NaN,56.090,55.780,NaN,NaN,NaN,NaN,NaN
1149,"Employment to population ratio, ages 15-24, to...",SL.EMP.1524.SP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.553,35.636,34.728,33.837,35.521,35.600,35.607,34.127,34.355,34.954
1251,"Employment to population ratio, ages 15-24, to...",SL.EMP.1524.SP.NE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.930,NaN,NaN,38.040,35.300,NaN,NaN,NaN,NaN,NaN


In [179]:
# Extract 'Employment to population ratio, 15+, total (%) (modeled ILO estimate) ' at row 684
df = extract_series(684)
df.tail()

Employment to population ratio, 15+, total (%) (modeled ILO estimate) 

                                      Indicator Name     Indicator Code  1960  \
0  Employment to population ratio, 15+, total (%)...  SL.EMP.TOTL.SP.ZS   NaN   

   1961  1962  1963  1964  1965  1966  1967  ...    2013    2014    2015  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  54.175  54.006  53.829   

    2016    2017    2018    2019    2020    2021    2022  
0  53.65  55.781  55.911  56.019  55.007  55.277  55.993  

[1 rows x 65 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    32 non-null     object 
 1   value   32 non-null     float64
dtypes: float64(1), object(1)
memory usage: 640.0+ bytes
None


,date,value
27,2018-01-01,55.911
28,2019-01-01,56.019
29,2020-01-01,55.007
30,2021-01-01,55.277
31,2022-01-01,55.993


In [181]:
# export to csv and json
df.to_csv('data/empl.csv', index=False)
df.to_json('data/empl.json', orient='records')

#### Employment

In [184]:
filter_series(raw_data, ['employment', 'total'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
55,"Share of youth not in education, employment or...",SL.UEM.NEET.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32.040000,NaN,NaN,28.880000,27.390000,NaN,NaN,NaN,NaN,NaN
63,Adequacy of unemployment benefits and ALMP (% ...,per_lm_alllm.adq_pop_tot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.881000,10.268000,10.666000,11.051000,12.219000,12.293000,12.366000,14.38300,13.772000,12.928
666,"Part time employment, total (% of total employ...",SL.TLF.PART.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.410000,NaN,NaN,18.690000,20.440000,NaN,NaN,NaN,NaN,NaN
677,Employment in services (% of total employment)...,SL.SRV.EMPL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.111920,35.063170,35.986890,36.875730,38.983370,39.909970,40.727910,40.87996,41.196040,NaN
680,Employment in industry (% of total employment)...,SL.IND.EMPL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.788890,20.550990,20.466740,20.459530,20.419330,20.666120,20.946730,21.24903,21.713960,NaN
681,"Contributing family workers, total (% of total...",SL.FAM.WORK.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.136140,16.937370,15.728230,14.508350,11.860110,11.111000,10.519300,10.47820,10.163300,NaN
684,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.175000,54.006000,53.829000,53.650000,55.781000,55.911000,56.019000,55.00700,55.277000,55.993
804,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.430000,NaN,NaN,4.350000,4.370000,NaN,NaN,NaN,NaN,NaN
806,Unemployment with intermediate education (% of...,SL.UEM.INTM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.980000,NaN,NaN,9.710000,8.480000,NaN,NaN,NaN,NaN,NaN


#### Participation Rate

In [190]:
filter_series(raw_data, ['participation rate', 'total', '15\+'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
812,"Labor force participation rate, total (% of to...",SL.TLF.CACT.NE.ZS,NaN,57.17,NaN,NaN,NaN,NaN,NaN,NaN,...,57.180,NaN,NaN,56.09,58.33,NaN,NaN,NaN,NaN,NaN
1231,"Labor force participation rate, total (% of to...",SL.TLF.CACT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,56.686,56.488,56.289,56.09,58.33,58.473,58.587,58.03,58.246,58.754


In [191]:
# Extract 'Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)' 
# at row 1231
df = extract_series(1231)
df.tail()

Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate) 

                                      Indicator Name  Indicator Code  1960  \
0  Labor force participation rate, total (% of to...  SL.TLF.CACT.ZS   NaN   

   1961  1962  1963  1964  1965  1966  1967  ...    2013    2014    2015  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  56.686  56.488  56.289   

    2016   2017    2018    2019   2020    2021    2022  
0  56.09  58.33  58.473  58.587  58.03  58.246  58.754  

[1 rows x 65 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    33 non-null     object 
 1   value   33 non-null     float64
dtypes: float64(1), object(1)
memory usage: 656.0+ bytes
None


,date,value
28,2018-01-01,58.473
29,2019-01-01,58.587
30,2020-01-01,58.030
31,2021-01-01,58.246
32,2022-01-01,58.754


In [192]:
# export to csv and json
df.to_csv('data/part.csv', index=False)
df.to_json('data/part.json', orient='records')

#### Labour Force

In [166]:
filter_series(raw_data, ['labor force', 'total'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
664,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.881000e+00,1.026800e+01,1.066600e+01,1.105100e+01,1.221900e+01,1.229300e+01,1.236600e+01,1.438300e+01,1.377200e+01,1.292800e+01
671,"Labor force participation rate, total (% of to...",SL.TLF.ACTI.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.821300e+01,5.804300e+01,5.788200e+01,5.773100e+01,6.040200e+01,6.064500e+01,6.086800e+01,6.035100e+01,6.065700e+01,NaN
672,"Labor force participation rate for ages 15-24,...",SL.TLF.ACTI.1524.NE.ZS,NaN,50.66,NaN,NaN,NaN,NaN,NaN,NaN,...,4.542000e+01,NaN,NaN,3.804000e+01,4.046000e+01,NaN,NaN,NaN,NaN,NaN
804,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.430000e+00,NaN,NaN,4.350000e+00,4.370000e+00,NaN,NaN,NaN,NaN,NaN
806,Unemployment with intermediate education (% of...,SL.UEM.INTM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.980000e+00,NaN,NaN,9.710000e+00,8.480000e+00,NaN,NaN,NaN,NaN,NaN
810,"Labor force, female (% of total labor force)",SL.TLF.TOTL.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.855937e+01,2.907011e+01,2.958657e+01,3.009696e+01,3.172093e+01,3.187737e+01,3.200891e+01,3.203826e+01,3.241967e+01,3.267183e+01
811,Labor force with intermediate education (% of ...,SL.TLF.INTM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.581000e+01,NaN,NaN,5.131000e+01,4.761000e+01,NaN,NaN,NaN,NaN,NaN
812,"Labor force participation rate, total (% of to...",SL.TLF.CACT.NE.ZS,NaN,57.17,NaN,NaN,NaN,NaN,NaN,NaN,...,5.718000e+01,NaN,NaN,5.609000e+01,5.833000e+01,NaN,NaN,NaN,NaN,NaN
816,"Labor force participation rate for ages 15-24,...",SL.TLF.ACTI.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.056100e+01,3.971400e+01,3.887400e+01,3.804100e+01,4.046500e+01,4.058900e+01,4.063200e+01,3.986000e+01,3.984200e+01,4.014400e+01
1125,Unemployment with advanced education (% of tot...,SL.UEM.ADVN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.000000e+00,NaN,NaN,9.030000e+00,1.074000e+01,NaN,NaN,NaN,NaN,NaN


In [172]:
# Extract 'Labor force, total' at row 1229
df = extract_series(1229)
df['value'] = df['value'].astype(int)
df.tail()

Labor force, total 

       Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  \
0  Labor force, total  SL.TLF.TOTL.IN   NaN   NaN   NaN   NaN   NaN   NaN   

   1966  1967  ...        2013        2014        2015        2016  \
0   NaN   NaN  ...  59781774.0  60904130.0  62009866.0  63166277.0   

         2017        2018        2019        2020        2021        2022  
0  67164738.0  68748318.0  70248864.0  70940010.0  72549161.0  74459362.0  

[1 rows x 65 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    33 non-null     object 
 1   value   33 non-null     float64
dtypes: float64(1), object(1)
memory usage: 656.0+ bytes
None


,date,value
28,2018-01-01,68748318
29,2019-01-01,70248864
30,2020-01-01,70940010
31,2021-01-01,72549161
32,2022-01-01,74459362


In [173]:
# export to csv and json
df.to_csv('data/labour_force.csv', index=False)
df.to_json('data/labour_force.json', orient='records')

---

#### Growth

In [149]:
filter_series(raw_data, ['growth'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
27,Rural population growth (annual %),SP.RUR.TOTL.ZG,NaN,2.755787,2.765277,2.765195,2.752662,2.731269,2.699307,2.721449,...,0.129949,0.089653,0.021228,0.044033,0.048708,-0.052658,-0.114188,-0.096784,-0.102357,-0.186068
75,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,NaN,3.019802,2.338669,-3.405414,7.668926,-1.391021,-0.437252,-4.781817,...,4.678722,4.747779,5.291061,5.803210,5.266455,6.080238,6.687654,2.271109,5.716649,5.954845
77,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,6.058161,5.453031,-0.455894,10.952789,1.606258,2.566812,-1.875864,...,6.013606,6.061059,6.552640,7.113478,6.590250,7.319413,7.881907,3.448026,6.938679,7.099829
94,"Manufacturing, value added (annual % growth)",NV.IND.MANF.KD.ZG,NaN,8.229668,7.004629,5.469353,4.940766,4.280426,6.052464,-0.028641,...,10.306890,8.766757,10.310577,11.690815,7.086440,10.452442,12.334487,1.680819,11.585139,11.410828
135,Claims on central government (annual growth as...,FM.AST.CGOV.ZG.M3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.453820,2.493596,2.372433,3.836338,3.279088,4.364018,6.278977,3.740638,4.016348,3.522746
283,Population growth (annual %),SP.POP.GROW,NaN,2.906641,2.997805,3.007812,3.004376,2.994281,2.972633,3.006246,...,1.267157,1.245960,1.191061,1.230795,1.249724,1.161378,1.113172,1.144210,1.149318,1.074837
304,Gross fixed capital formation (annual % growth),NE.GDI.FTOT.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.364642,9.855051,7.118801,8.909372,8.359574,12.132901,6.880267,3.945884,8.090779,11.653373
306,Exports of goods and services (annual % growth),NE.EXP.GNFS.KD.ZG,NaN,-11.501106,20.796544,4.830609,3.742608,-5.189872,14.332446,-11.766218,...,2.451884,3.201149,-2.829990,2.196750,-1.843281,6.105920,11.456473,-17.502392,9.191456,29.392555
309,Final consumption expenditure (annual % growth),NE.CON.TOTL.KD.ZG,NaN,8.949271,-1.743946,4.911219,10.593545,0.481668,1.841663,-2.947137,...,5.173398,4.275974,6.028725,3.399133,6.344413,9.077003,5.525803,2.908652,7.920388,7.374349
367,GNI growth (annual %),NY.GNP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.597280,4.692012,6.354338,5.395267,5.315838,7.925879,7.907855,3.524300,8.065671,5.802404


In [150]:
# Extract 'GDP growth (annual %)' at row 77
df = extract_series(77)
df.tail()

GDP growth (annual %) 

          Indicator Name     Indicator Code  1960      1961      1962  \
0  GDP growth (annual %)  NY.GDP.MKTP.KD.ZG   NaN  6.058161  5.453031   

       1963       1964      1965      1966      1967  ...      2013      2014  \
0 -0.455894  10.952789  1.606258  2.566812 -1.875864  ...  6.013606  6.061059   

      2015      2016     2017      2018      2019      2020      2021  \
0  6.55264  7.113478  6.59025  7.319413  7.881907  3.448026  6.938679   

       2022  
0  7.099829  

[1 rows x 65 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    62 non-null     object 
 1   value   62 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.1+ KB
None


,date,value
57,2018-01-01,7.319413
58,2019-01-01,7.881907
59,2020-01-01,3.448026
60,2021-01-01,6.938679
61,2022-01-01,7.099829


In [152]:
# export
df.to_csv('data/grow.csv', index=False)
df.to_json('data/grow.json', orient='records')

`ToDo`: check whether we want percentage values in 100 scale or 1 scale
- 100 scale for now

---

#### Inflation

In [155]:
filter_series(raw_data, ['Inflation, consumer prices'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
737,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.530406,6.991639,6.19428,5.513526,5.70207,5.543621,5.591996,5.691075,5.545654,7.696954


In [156]:
# Extract 'Inflation, consumer prices (annual %) ' at row 737
df = extract_series(737)
df.tail()

Inflation, consumer prices (annual %) 

                          Indicator Name  Indicator Code  1960  1961  1962  \
0  Inflation, consumer prices (annual %)  FP.CPI.TOTL.ZG   NaN   NaN   NaN   

   1963  1964  1965  1966  1967  ...      2013      2014     2015      2016  \
0   NaN   NaN   NaN   NaN   NaN  ...  7.530406  6.991639  6.19428  5.513526   

      2017      2018      2019      2020      2021      2022  
0  5.70207  5.543621  5.591996  5.691075  5.545654  7.696954  

[1 rows x 65 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36 non-null     object 
 1   value   36 non-null     float64
dtypes: float64(1), object(1)
memory usage: 704.0+ bytes
None


,date,value
31,2018-01-01,5.543621
32,2019-01-01,5.591996
33,2020-01-01,5.691075
34,2021-01-01,5.545654
35,2022-01-01,7.696954


In [157]:
# export
df.to_csv('data/infl.csv', index=False)
df.to_json('data/infl.json', orient='records')

---

#### Inequality

In [158]:
filter_series(raw_data, ['gini'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
695,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,32.4,NaN,NaN,NaN,NaN,NaN,31.8


In [159]:
# Extract 'Gini index' at row 695
df = extract_series(695)
df.tail()

Gini index 

  Indicator Name Indicator Code  1960  1961  1962  1963  1964  1965  1966  \
0     Gini index    SI.POV.GINI   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   1967  ...  2013  2014  2015  2016  2017  2018  2019  2020  2021  2022  
0   NaN  ...   NaN   NaN   NaN  32.4   NaN   NaN   NaN   NaN   NaN  31.8  

[1 rows x 65 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    10 non-null     object 
 1   value   10 non-null     float64
dtypes: float64(1), object(1)
memory usage: 288.0+ bytes
None


,date,value
5,2000-01-01,33.4
6,2005-01-01,33.2
7,2010-01-01,32.1
8,2016-01-01,32.4
9,2022-01-01,31.8


In [160]:
# export
df.to_csv('data/ineq.csv', index=False)
df.to_json('data/ineq.json', orient='records')

---

#### Exports

In [153]:
filter_series(raw_data, ['exports'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
3,High-technology exports (% of manufactured exp...,TX.VAL.TECH.MF.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.990886e-01,NaN,3.078671e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.R6.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.715582e-01,3.722408e-01,4.591402e-01,3.795023e-01,5.308626e-01,5.061223e-01,4.426419e-01,5.483284e-01,NaN,NaN
6,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.R2.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.417100e+00,3.319974e+00,2.859091e+00,2.866961e+00,3.038562e+00,2.410128e+00,2.385277e+00,2.971103e+00,NaN,NaN
7,Merchandise exports (current US$),TX.VAL.MRCH.CD.WT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.911400e+10,3.040500e+10,3.237900e+10,3.489400e+10,3.585100e+10,3.925200e+10,3.933700e+10,3.360500e+10,4.422300e+10,5.469500e+10
8,Insurance and financial services (% of commerc...,TX.VAL.INSF.ZS.WT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.115775e+00,4.421320e+00,4.241769e+00,4.474893e+00,6.327005e+00,3.755721e+00,4.640008e+00,3.296103e+00,2.719318e+00,3.547923e+00
9,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.051364e+00,NaN,7.471825e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,Taxes on exports (current LCU),GC.TAX.EXPT.CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.991000e+08,3.350000e+07,3.910000e+07,3.000000e+08,2.164000e+08,3.388000e+08,1.145572e+09,7.739000e+08,5.889668e+06,NaN
164,Public and publicly guaranteed debt service (%...,DT.TDS.DPPG.XP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.852805e+00,4.857939e+00,3.689619e+00,3.253705e+00,3.786378e+00,3.837185e+00,4.223391e+00,5.204949e+00,5.585951e+00,NaN
234,"Transport services (% of service exports, BoP)",BX.GSR.TRAN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.656677e+01,1.251030e+01,1.241368e+01,1.180547e+01,1.262651e+01,1.203831e+01,1.000412e+01,1.138530e+01,1.786736e+01,1.746052e+01
235,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.863819e+10,2.992449e+10,3.173604e+10,3.412205e+10,3.530077e+10,3.868190e+10,3.874729e+10,3.245615e+10,4.181597e+10,5.186827e+10


---

#### Productivity

In [185]:
filter_series(raw_data, ['productivity'])

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
141,"Water productivity, total (constant 2015 US$ G...",ER.GDP.FWTL.M3.KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.812359,5.104039,5.438488,5.825354,6.209259,6.663741,7.18897,7.436848,NaN,NaN
